<a href="https://colab.research.google.com/github/VGoma23/data-520-asean/blob/main/Sentinel2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='iuu-fishing-detections-asean')

In [ ]:
m = geemap.Map(center=[9.4, 107.2], zoom=20, height=600)

In [ ]:
vietnam_coast = ee.Geometry.Polygon([[
 [108.075256, 9.925566],
 [108.075256, 8.619041],
 [107.482239, 8.619041],
 [107.482239, 9.925566]]])

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(vietnam_coast)
    .filterDate('2020-01-01', '2025-10-29')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m.add_layer(dataset.median(), visualization, 'Historic Sentinel')
image = dataset.first()
m.add_layer(image, visualization, 'sentinel')
# image = dataset.sort('system:time_start', False).first().clip(vietnam_coast)
# m.add_layer(image, visualization, 'sentinel')
m

Map(bottom=31218.0, center=[10.109486058403773, 108.52844238281251], controls=(WidgetControl(options=['positio…

In [ ]:
image